<a href="https://colab.research.google.com/github/JS0501/ESAA_OB/blob/main/09_12_%EC%84%B8%EC%85%98_%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


확률적 경사하강법 또는 미니 배치 경사하강법을 사용하여 적은 메모리를 사용하여 빠르게 모델을 훈련할 수 있다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

모델이 과적합 되었다. sample size를 늘리거나 hyperparameter를 조절한다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

높은 분산이 문제이다. 알파를 줄여야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀

릿지를 사용하여 규제를 주면, 평범한 선형 회귀보다 모델의 과적합을 막을 수 있다.

- 릿지 회귀 대신 라쏘 회귀

라쏘 회귀를 사용하여 특성을 선택할 수 있다.

- 라쏘 회귀 대신 엘라스틱넷

릿지와 라쏘가 결합된 엘라스틱넷을 사용하면 특성 수가 훈련 샘플 수보다 많거나 특성 간 강한 상관관계가 있을 때 도움이 된다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [ ]:
import numpy as np

# 주어진 데이터
from sklearn import datasets
iris = datasets.load_iris()
X = iris["data"][:,3:]  # petal width 만 사용
y = iris["target"]  # Use the full target array

# train / val 분리
np.random.seed(0)
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]
n_train = int(0.6 * len(X))
n_val   = int(0.2 * len(X))
X_train, y_train = X[:n_train], y[:n_train]
X_val,   y_val   = X[n_train:n_train+n_val], y[n_train:n_train+n_val]

# Convert y to binary *after* splitting
y_train = (y_train == 2).astype(int)  # Virginica 여부 바이너리
y_val = (y_val == 2).astype(int)

# 편향 항(bias) 추가
X_train = np.hstack([X_train, np.ones((X_train.shape[0],1))])
X_val   = np.hstack([X_val,   np.ones((X_val.shape[0],1))])

# 시그모이드, 손실
def sigmoid(z): return 1 / (1 + np.exp(-z))
def loss(X, y, w):
    z = X.dot(w)
    p = sigmoid(z)
    # 이항 크로스엔트로피
    return - np.mean(y * np.log(p + 1e-15) + (1 - y) * np.log(1 - p + 1e-15))

# 학습
w = np.zeros(X_train.shape[1])
lr = 0.5
epochs = 1000
patience = 10
best_w = None
best_loss = float('inf')
wait = 0

for epoch in range(epochs):
    # 전체 배치 경사 하강 (batch GD)
    p = sigmoid(X_train.dot(w))
    grad = X_train.T.dot(p - y_train) / len(y_train)  # Use y_train here
    w -= lr * grad

    l_val = loss(X_val, y_val, w)
    if l_val < best_loss - 1e-6:
        best_loss = l_val
        best_w = w.copy()
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping at epoch", epoch)
            break

# 최종 파라미터는 best_w
w = best_w
print("Validation loss:", best_loss)

Validation loss: 0.07829856578039335
